In [138]:
import os
import json
import math
import numpy as np
from scipy.spatial.transform import Rotation

In [139]:
SAVE_PATH = os.path.join(os.getcwd(), '../data/hemisphere')
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
if not os.path.exists(os.path.join(SAVE_PATH, 'images')):
    os.makedirs(os.path.join(SAVE_PATH, 'images'))

In [140]:
transforms = {}
transforms["fl_x"] = 762.72 # focal length in x
transforms["fl_y"] = 762.72# focal length in y
transforms["w"] = 1280 # image width
transforms["h"] = 720 # image height
transforms["cx"] = transforms["w"]//2 # optical center in x
transforms["cy"] = transforms["h"]//2 # optical center in y
transforms["k1"] = 0
transforms["k2"] = 0
transforms["k3"] = 0
transforms["p1"] = 0
transforms["p2"] = 0
transforms["camera_model"] = "OPENCV" # camera model

In [141]:
frames = []

r = 15

cnt = 0

for i in range(-90, 90, 15):
    for j in range(-90, 90, 15):
        transform_matrix = np.eye(4)

        # Translation
        transform_matrix[:3, 3] = [0, 0, r]

        # Rotation
        rot = Rotation.from_euler('xyz', [i, 0, j], degrees=True)
        rot_matrix = np.eye(4)
        rot_matrix[:3, :3] = rot.as_matrix()
        transform_matrix = np.matmul(rot_matrix, transform_matrix)

        frame = {}
        file_path = "images/{}.png".format(cnt)

        frame["file_path"] = file_path
        frame["transform_matrix"] = transform_matrix.tolist()

        frames.append(frame)
        cnt+=1

transforms["frames"] = frames

[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0e-05  1.5e+01]
[-1.0e-04 -1.0

In [142]:
#Save transforms
with open(os.path.join(SAVE_PATH,'transforms.json'), 'w') as outfile:
    json.dump(transforms, outfile, indent=4)